In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime, timedelta

In [4]:
def extract_links_by_class(url):
    driver=webdriver.Chrome() # set the driver
    driver.get(url) # open webdriver
    driver.maximize_window() # maximize 
    
# In this part for Using Key element scrolldown the website

    driver.find_element(By.TAG_NAME,"body").send_keys(Keys.PAGE_DOWN)
    time.sleep(1) # waiting time 1 sceond

# Find a tag with By method and XPATH
    view_all = driver.find_elements(By.XPATH,"//a[@class='OfferSection__ViewAllText-sc-16xojcc-1 eVcjqm']") # find a tag class and paste in here 
    ref=view_all[1].get_attribute('href') # Extract href from a tag
    time.sleep(2)
    driver.get(ref) # open link Extract from href tag

    links = driver.find_elements(By.XPATH,"//a[@class='D113_link']") # Find all state link using XPATH 

# Extract all state name and href link from a tag
    state_route_links = []
    for link in links: 
        state = link.text # Extract state
        href = link.get_attribute("href") # Extract href link
        if href:
            route_links = href
        state_route_links.append((state, route_links))
    driver.quit() # close drive
    return state_route_links

url = "https://www.redbus.in/" # link for open drive

# call the function and store the data in route_links
route_links  = extract_links_by_class(url) 

In [5]:
state = {'PEPSU (Punjab)':"Punjab",
         'Himachal Pradesh Tourism Development Corporation (HPTDC)':"Himachal Pradesh",
         'RSRTC':"Rajasthan",
         'PEPSU (Punjab)':"Punjab",
         'UPSRTC':"uttar pradesh",
         'Chandigarh Transport Undertaking (CTU)':"Chandigarh",
         'HRTC':"Himachal Pradesh",
         'BSRTC Operated By VIP Travels':"Kathmandu",
         'South Bengal State Transport Corporation (SBSTC)':"South Bengal",
         'NORTH BENGAL STATE TRANSPORT CORPORATION':"North BenGal",
         'Bihar State Tourism Development Corporation (BSTDC)':"Bihar",
         'WBTC (CTC)':"West Bengal",
         'West Bengal Transport Corporation':"West Bengal",
         'Bihar State Road Transport Corporation (BSRTC)':"Bihar",
         'Sikkim Nationalised Transport (SNT)':"Sikkim",
         'Assam State Transport Corporation (ASTC)':"Assam",
         'KAAC TRANSPORT':"Assam",
         'Meghalaya Transport Corporation(MTC)':"Meghalaya",
         'Kadamba Transport Corporation Limited (KTCL)':"Goa",
         'TNSTC':"Tamil Nadu",
         'APSRTC':"Andhra Pradesh",
         'Puducherry Road Transport Corporation (PRTC)':"Puducherry",
         'TSRTC':"Telangana",
         'KSRTC (Kerala)':"Kerala",
         'GSRTC':"Gujarat",
         }
state_rout_link = [] 
for i, k in route_links:
    state_rout_link.append((state[i],k))

In [6]:
def extract_links_by_class(state, url, class_name):
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()

    routes = []

# route_function() for Extract all bus route in one page

    def route_function(): 
        route_elements = driver.find_elements(By.CLASS_NAME, class_name)
        routes_ = [] 
        for route_element in route_elements:
            route_ = route_element.text  
            route_link_ = route_element.get_attribute('href')
            routes_.append((state, route_, route_link_))
        return routes_
# scrolldown the page and click the next page
    try:
        body_element = driver.find_element(By.TAG_NAME, "body") # find body element for scrolldown 
        for _ in range(3): # scrolldown 3 time 
            body_element.send_keys(Keys.PAGE_DOWN) 
            time.sleep(1)          
        view_buses_button = driver.find_elements(By.CLASS_NAME, 'DC_117_pageTabs ') # find all nextpage button like page 2, page 3, page 4
# navigate to next pages
        for i in range(len(view_buses_button)):
            if view_buses_button:
                view_buses_button[i].click()
                routes.append(route_function())
            else:
                print("No button found")

        driver.quit()
            
    except Exception as e:
        print("Error occurred:", e)
        driver.quit()
    return routes

# extract all bus routes
routes = []
for state, link in state_rout_link:
    div_locator_1 = "route" 
    routes.append(extract_links_by_class(state, link, div_locator_1)) 

In [7]:

dic = {}
bus_routes = []
for i in routes:
    for j in i:
        for k,l,p in j:
            if k not in dic:
                dic[k] = 1
            else:
                dic[k] += 1
            if dic[k]<= 11:
                  bus_routes.append((k,l,p))

len(bus_routes) # find count of route

165

In [8]:
# set for date and time (Optional)
"""def convert_to_datetime(time_str, reference_date):
    try:
        dt = datetime.strptime(time_str, '%H:%M').replace(year=reference_date.year, month=reference_date.month, day=reference_date.day)
        return dt
    except ValueError:
        return None"""

count = 0 # count of route link
bus_details = []
for state, route, route_link in bus_routes:
            try:
                driver = webdriver.Chrome()
                driver.get(route_link)
                driver.maximize_window()
                time.sleep(5) # waiting for loading time of web page
                view_buses_button = driver.find_element(By.CLASS_NAME, 'button') # view bus button for view government buses
                view_buses_button.click()
                time.sleep(1)

                body_element = driver.find_element(By.TAG_NAME, "body") 
                for _ in range(20): # scrolldown 20 times for extract all data 
                    body_element.send_keys(Keys.PAGE_DOWN)          
                time.sleep(2)  # Wait to load the page


            except:
                pass 
# Find all the bus detail element 

            try:
                bus_elements = driver.find_elements(By.CSS_SELECTOR, "div.bus-item")
            except:
                print("No bus elements found")
                continue
# extract all bus data one by one 

            for bus in bus_elements:
                try:
                    busname = bus.find_element(By.CSS_SELECTOR, "div.travels.lh-24.f-bold.d-color").text
                except:
                    busname = "N/A"

                try:
                    bustype = bus.find_element(By.CSS_SELECTOR, "div.bus-type.f-12.m-top-16.l-color.evBus").text
                except:
                    bustype = "N/A"

                try:
                    departing_time = bus.find_element(By.CSS_SELECTOR, "div.dp-time.f-19.d-color.f-bold").text
                    # departing_time_dt = convert_to_datetime(departing_time, datetime.now())
                except:
                    departing_time_dt = None

                try:
                    duration = bus.find_element(By.CSS_SELECTOR, "div.dur.l-color.lh-24").text
                except:
                    duration = "N/A"
                try:
                    reaching_time = bus.find_element(By.CSS_SELECTOR, "div.bp-time.f-19.d-color.disp-Inline").text
                    # reaching_time_dt = convert_to_datetime(reaching_time, datetime.now())
                    if reaching_time_dt and departing_time_dt and reaching_time_dt < departing_time_dt:
                        reaching_time_dt += timedelta(days=1)
                except:
                    reaching_time_dt = None

                try:
                    star_rating = bus.find_element(By.CSS_SELECTOR, "div.rating-sec.lh-24").text
                    star_rating = float(star_rating) if star_rating != "N/A" else 0.0
                except:
                    star_rating = 0.0

                try:
                    price = bus.find_element(By.CSS_SELECTOR, "span.f-19.f-bold").text
                    price = float(price.replace('₹', '').replace(',', '').strip()) if price != "N/A" else None
                except:
                    price = None

                try:
                    try:
                        seats_available = bus.find_element(By.CSS_SELECTOR, "div.seat-left.m-top-16").text
                    except:
                        seats_available = bus.find_element(By.CSS_SELECTOR, "div.seat-left.m-top-30").text

                    seats_available = int(seats_available.split()[0]) if seats_available != "N/A" else 0
                except:
                        seats_available = 0
                bus_details.append((route, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available, state)) # append all the data to bus_details
            count+=1
            print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165


In [9]:
import pandas as pd

columns = ['route', 'route_link', 'busname', 'bustype','departing_time','duration','reaching_time','star_rating','price','seats_available', 'state']
redbus_data =  pd.DataFrame(bus_details,columns=columns)

In [10]:
From = []
To = []
location_data = redbus_data['route']
for i in location_data:
    From_1, To_1 = i.split(" to ")
    From.append(From_1)
    To.append(To_1)
redbus_data["From"] = From
redbus_data["To"] = To

In [11]:
redbus_data.to_csv("redbus_data.csv")

In [12]:
redbus_data=pd.read_csv("redbus_data.csv")

In [13]:
redbus_data

,Unnamed: 0,route,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available,state,From,To
0,0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 193271,Express Non AC Seater 2+3,19:30,06h 30m,02:00,4.2,294.0,40,Rajasthan,Udaipur,Jodhpur
1,1,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,Shree Devnarayan Travels,Bharat Benz A/C Seater (2+1),17:00,04h 30m,21:30,4.5,550.0,10,Rajasthan,Udaipur,Jodhpur
2,2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,M R Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),17:10,04h 45m,21:55,4.4,600.0,39,Rajasthan,Udaipur,Jodhpur
3,3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,AdKalpana Travels,NON AC Seater / Sleeper 2+1,22:00,06h 54m,04:54,3.6,300.0,21,Rajasthan,Udaipur,Jodhpur
4,4,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,Raj Travel,A/C Sleeper (2+1),22:30,07h 00m,05:30,4.6,560.0,18,Rajasthan,Udaipur,Jodhpur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,3118,Guwahati to Shillong,https://www.redbus.in/bus-tickets/guwahati-to-...,NETWORK TRAVELS,NON A/C Seater (2+2),16:15,03h 15m,19:30,3.2,300.0,40,Meghalaya,Guwahati,Shillong
3119,3119,Tura (Meghalaya) to Shillong,https://www.redbus.in/bus-tickets/tura-meghala...,NETWORK TRAVELS,NON A/C Seater (2+2),19:00,13h 00m,08:00,3.2,650.0,24,Meghalaya,Tura (Meghalaya),Shillong
3120,3120,Shillong to Silchar,https://www.redbus.in/bus-tickets/shillong-to-...,SEEMA TRAVELS,NON AC Seater / Sleeper 2+1,20:30,08h 30m,05:00,1.0,650.0,29,Meghalaya,Shillong,Silchar
3121,3121,Silchar to Shillong,https://www.redbus.in/bus-tickets/silchar-to-s...,Meghalaya Transport Corporation(MTC) - 163362,Tata Non A/C Seater Executive Luxury (2+2),18:00,09h 30m,03:30,2.8,384.0,7,Meghalaya,Silchar,Shillong


In [14]:
!pip install sqlalchemy

In [15]:
!pip install mysql-connector-python

In [16]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import mysql.connector

# Your MySQL connection
#mydb = mysql.connector.connect(
    #host="localhost",
    #user="root",
    #password="",
#)

# Create a SQLAlchemy engine
engine = create_engine('mysql+mysqlconnector://root:@localhost/local.1')

try:
    redbus_data.to_sql('redbus_data', con=engine, if_exists='replace', index=False)
    print("Data uploaded successfully!")

except SQLAlchemyError as e:
    print(f"An error occurred: {e}")

Data uploaded successfully!
